In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from environment import *
from setting import *

In [ ]:
feature_x_sample = pd.read_table(PATH_DICT["feature_x_sample_file_path"], index_col=0)

In [ ]:
feature_x_sample__normalized = pd.DataFrame(
    ccal.normalize_nd_array(feature_x_sample.values, None, "0-1", raise_for_bad=False),
    index=feature_x_sample.index,
    columns=feature_x_sample.columns,
)

In [ ]:
if SELECT_FEATURE_AUTOMATICALLY:

    features = ccal.select_series_low_and_high_index(
        feature_x_sample__normalized.std(axis=1),
        title="Feature Raw Signal Standard Deviation",
    )[1]

else:

    features = feature_x_sample.index

print("Selected {} feature.".format(features.size))

In [ ]:
if SELECT_SAMPLE_AUTOMATICALLY:

    samples = ccal.select_series_low_and_high_index(
        feature_x_sample__normalized.std(), title="Sample Raw Signal Standard Deviation"
    )[1]

else:

    samples = feature_x_sample.columns

print("Selected {} sample.".format(samples.size))

In [ ]:
signal = feature_x_sample__normalized.loc[features, samples]

In [ ]:
signal.to_csv(PATH_DICT["raw_signal_matrix_file_path"], sep="\t")

ccal.summarize_feature_x_sample(
    signal,
    title="Raw Signal Matrix",
    feature_name=FEATURE_NAME,
    sample_name=SAMPLE_NAME,
)

signal

In [ ]:
if "feature" in ELEMENTS:

    feature_context = pd.read_table(
        PATH_DICT["feature_context_matrix_file_path"], index_col=0
    )

    feature_negative_signal = -feature_context.clip(upper=0)

    feature_positive_signal = feature_context.clip(lower=0)

In [ ]:
if "sample" in ELEMENTS:

    sample_context = pd.read_table(
        PATH_DICT["sample_context_matrix_file_path"], index_col=0
    )

    sample_negative_signal = -sample_context.clip(upper=0).T

    sample_positive_signal = sample_context.clip(lower=0).T

In [ ]:
if "feature" in ELEMENTS and "sample" in ELEMENTS:

    negative_signal = feature_negative_signal + sample_negative_signal

    positive_signal = feature_positive_signal + sample_positive_signal

elif "feature" in ELEMENTS:

    negative_signal = feature_negative_signal

    positive_signal = feature_positive_signal

elif "sample" in ELEMENTS:

    negative_signal = sample_negative_signal

    positive_signal = sample_positive_signal

In [ ]:
if SELECT_FEATURE_AUTOMATICALLY:

    negative_features = ccal.select_series_low_and_high_index(
        negative_signal.sum(axis=1), title="Feature Negative Signal Sum"
    )[1]

    positive_features = ccal.select_series_low_and_high_index(
        positive_signal.sum(axis=1), title="Feature Positive Signal Sum"
    )[1]

else:

    negative_features = negative_signal.index

    positive_features = positive_signal.index

print(
    "Selected {} negative and {} positive feature.".format(
        negative_features.size, positive_features.size
    )
)

In [ ]:
if SELECT_SAMPLE_AUTOMATICALLY:

    negative_samples = ccal.select_series_low_and_high_index(
        negative_signal.sum(), title="sample Negative Signal Sum"
    )[1]

    positive_samples = ccal.select_series_low_and_high_index(
        positive_signal.sum(), title="sample Positive Signal Sum"
    )[1]

else:

    negative_samples = negative_signal.columns

    positive_samples = positive_signal.columns

print(
    "Selected {} negative and {} positive sample.".format(
        negative_samples.size, positive_samples.size
    )
)

In [ ]:
negative_signal__selected = negative_signal.loc[negative_features, negative_samples]

positive_signal__selected = positive_signal.loc[positive_features, positive_samples]

In [ ]:
if "negative" in CONTEXTS and "positive" in CONTEXTS:

    negative_signal__selected.index = (
        "(-) {}".format(feature) for feature in negative_signal__selected.index
    )

    positive_signal__selected.index = (
        "(+) {}".format(feature) for feature in positive_signal__selected.index
    )

    signal = pd.concat((negative_signal__selected, positive_signal__selected))

elif "negative" in CONTEXTS:

    signal = negative_signal__selected

elif "positive" in CONTEXTS:

    signal = positive_signal__selected

In [ ]:
if "feature" in ELEMENTS and "sample" in ELEMENTS:

    axis = None

elif "feature" in ELEMENTS:

    axis = 1

elif "sample" in ELEMENTS:

    axis = 0

signal = pd.DataFrame(
    ccal.normalize_nd_array(signal.values, axis, "0-1", raise_for_bad=False),
    index=signal.index,
    columns=signal.columns,
)

In [ ]:
signal.to_csv(PATH_DICT["context_signal_matrix_file_path"], sep="\t")

ccal.summarize_feature_x_sample(
    signal, title="Signal Matrix", feature_name=FEATURE_NAME, sample_name=SAMPLE_NAME
)

signal